In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from fbprophet import Prophet
import pandas as pd

### Initialize a Spark session that connects to MongoDB

In [ ]:
spark = SparkSession \
    .builder \
    .appName("SentimentTimeSeries") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1:27017/mydb.mycollection") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1:27017/mydb.mycollection") \
    .getOrCreate()

### Reading data from MongoDB

In [ ]:


df = spark.read.format("mongo").load()

### Sentiment analysis preprocessing pipeline

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)

### Assemble pipeline

In [ ]:

pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, lr])

### Train the sentiment analysis model

In [ ]:
model = pipeline.fit(df)

### Assume the sentiment data is already in 'df' with 'timestamp' and 'sentiment' columns
#### Select relevant columns and convert to pandas for Prophet

In [ ]:
pandas_df = df.select("timestamp", "sentiment").withColumnRenamed("timestamp", "ds").withColumnRenamed("sentiment", "y").toPandas()

### Initialize and fit the Prophet model for time series forecasting

In [ ]:
prophet_model = Prophet()
prophet_model.fit(pandas_df)

### Create future dataframe for forecasting

In [ ]:

future_week = prophet_model.make_future_dataframe(periods=7)
future_month = prophet_model.make_future_dataframe(periods=30)
future_3month = prophet_model.make_future_dataframe(periods=90)

#### Perform forecasts

In [ ]:

forecast_week = prophet_model.predict(future_week)
forecast_month = prophet_model.predict(future_month)
forecast_3month = prophet_model.predict(future_3month)

### The forecasts can now be used or saved back to MongoDB
### This is just an example of converting a forecast back to a Spark DataFrame and saving it

In [ ]:
forecast_week_df = spark.createDataFrame(forecast_week)
forecast_week_df.write.format("mongo").mode("overwrite").option("collection", "forecast_week_collection").save()

### Close the Spark session

In [ ]:
spark.stop()